# Exploratory Testing Notebook for Asset Time Machine App

In [1]:
# base packages
import numpy as np
import pandas as pd

## Testing Data Connections

### Pulling from Yahoo Finance's API

In [2]:
import fix_yahoo_finance as yf

# pulling historical data for Apple using fixed yahoo finance API
initial_data_pull_df = yf.download(
    tickers='AAPL',
    start='2016-01-01',
    end='2018-01-01',
    progress=False
)

initial_data_pull_df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-12-22,174.68,175.42,174.50,175.01,171.03,16114600
2017-12-26,170.80,171.47,169.68,170.57,166.69,33185500
2017-12-27,170.10,170.78,169.71,170.60,166.72,21498200
2017-12-28,171.00,171.85,170.48,171.08,167.19,16480200
2017-12-29,170.52,170.59,169.22,169.23,165.38,25884400


## Data Processing

### Trimming to Only Needed Data

In [3]:
# making date a column instead of an index
initial_data_pull_df.reset_index(level=0, inplace=True)

# keeping just date and adjusted closing value
cleaned_close_df = initial_data_pull_df.loc[:,('Date','Adj Close')]

In [4]:
cleaned_close_df.tail()

,Date,Adj Close
498,2017-12-22,171.03
499,2017-12-26,166.69
500,2017-12-27,166.72
501,2017-12-28,167.19
502,2017-12-29,165.38


### Finding % Down from ATH for Each Value

In [17]:
# need to find an efficient and pythonic way to, for each date, find the % down from all-time high

# instantiate empty df in which to store ATH details
ath_info_df = pd.DataFrame(columns=['recent_ath_date', 'recent_ath_val', 'date_of_interest'])

for date in cleaned_close_df.loc[:,'Date']:
    
    # create df of time period to evaluate (all closes prior to chosen date)
    period_to_eval_df = cleaned_close_df.loc[cleaned_close_df.loc[:,'Date'] < date, :]

    # check the period to evaluate for ATH and get the date and value
    recent_ath_row = period_to_eval_df.loc[period_to_eval_df.loc[:,'Adj Close'] == period_to_eval_df.loc[:,'Adj Close'].max(), :]
    # store date of present iteration
    recent_ath_row['date_of_interest'] = date
    # rename columns so as to not be confusing
    recent_ath_row.rename(index=str, columns={"Date": "recent_ath_date", "Adj Close": "recent_ath_val"}, inplace=True)
    # append the ATH info for present iteration to the df of all ATH infos
    ath_info_df = ath_info_df.append(recent_ath_row)

/Users/pauljeffries/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [19]:
ath_info_df.tail()

,recent_ath_date,recent_ath_val,date_of_interest
494,2017-12-18,172.4,2017-12-22
494,2017-12-18,172.4,2017-12-26
494,2017-12-18,172.4,2017-12-27
494,2017-12-18,172.4,2017-12-28
494,2017-12-18,172.4,2017-12-29


In [28]:
# merging the main df and the df with ATH info
merged_df = pd.merge(
    cleaned_close_df, 
    ath_info_df,  
    left_on=['Date'], 
    right_on=['date_of_interest']
)

In [24]:
merged_df.tail()

,Date,Adj Close,recent_ath_date,recent_ath_val,date_of_interest
498,2017-12-22,171.03,2017-12-18,172.4,2017-12-22
499,2017-12-26,166.69,2017-12-18,172.4,2017-12-26
500,2017-12-27,166.72,2017-12-18,172.4,2017-12-27
501,2017-12-28,167.19,2017-12-18,172.4,2017-12-28
502,2017-12-29,165.38,2017-12-18,172.4,2017-12-29


In [25]:
?pd.merge

Signature:
pd.merge(
    left,
    right,
    how='inner',
    on=None,
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=False,
    suffixes=('_x', '_y'),
    copy=True,
    indicator=False,
    validate=None,
)
Docstring:
Merge DataFrame or named Series objects with a database-style join.

The join is done on columns or indexes. If joining columns on
columns, the DataFrame indexes *will be ignored*. Otherwise if joining indexes
on indexes or indexes on a column or columns, the index will be passed on.

Parameters
----------
left : DataFrame
right : DataFrame or named Series
    Object to merge with.
how : {'left', 'right', 'outer', 'inner'}, default 'inner'
    Type of merge to be performed.

    * left: use only keys from left frame, similar to a SQL left outer join;
      preserve key order.
    * right: use only keys from right frame, similar to a SQL right outer join;
      preserve key order.
    * outer: use union of keys from both frame

In [ ]:
recent_ath_row.loc[:,'Adj Close'].values[0]

In [ ]:
# adding ATH info to main df
cleaned_close_df['recent_ath_date'] = recent_ath_date

In [ ]:
cleaned_close_df.head()